In [23]:
import tkinter as tk
from tkinter import *

from tkinter import filedialog, messagebox, simpledialog

import pandas as pd
import numpy as np

from pandastable import Table

janela = Tk()
janela.title("Editor de Excel com Pandas")
janela.attributes("-fullscreen", True)

class ExcelEditor:
    
    def __init__(self, janela_principal):
        self.janela_principal = janela_principal
        self.resultado_label = Label(self.janela_principal, text="Total: ", font="Arial 16", bg="#F5F5F5")
        self.resultado_label.pack(side=TOP, padx=10, pady=10)

        # Criar um dataframe
        self.df = pd.DataFrame()
        
        # INICIANDO VÁRIAVEIS
        self.tree = None
        self.table = None
        self.filename = ""
        
        # CRIANDO OS WIDGETS
        self.cria_widgtes()
    
    def cria_widgtes(self):
        
        # CRIA JANELA MENU
        menu_bar = tk.Menu(self.janela_principal)
        
        # CRIA MENU ARQUIVO
        menu_de_arquivos = tk.Menu(menu_bar, tearoff=0)
        menu_de_arquivos.add_command(label="Abrir", command=self.carregar_excel)
        menu_de_arquivos.add_separator()
        menu_de_arquivos.add_command(label="Salvar como", command=janela.destroy)
        menu_de_arquivos.add_separator()
        menu_de_arquivos.add_command(label="Sair", command=janela.destroy)
       
        menu_bar.add_cascade(label="Arquivo", menu=menu_de_arquivos)
              
        # CRIA MENU EDITAR
        menu_edicao = tk.Menu(menu_bar, tearoff=0)
        menu_edicao.add_command(label="Renomear Coluna", command=self.renomear_coluna)
        menu_edicao.add_command(label="Remover Coluna", command=janela.destroy)
        menu_edicao.add_command(label="Filtrar", command=janela.destroy)
        menu_edicao.add_command(label="Pivot", command=janela.destroy)
        menu_edicao.add_command(label="Group", command=janela.destroy)
        menu_edicao.add_command(label="Remover linhas em branco", command=janela.destroy)
        menu_edicao.add_command(label="Remover linhas alternadas", command=janela.destroy)
        menu_edicao.add_command(label="Remover Duplicados", command=janela.destroy)
        menu_bar.add_cascade(label="Editar", menu=menu_edicao) 

        # CRIA MENU MERGE
        merge_menu = tk.Menu(menu_bar, tearoff=0)
        merge_menu.add_command(label="Inner Join", command=janela.destroy)
        merge_menu.add_command(label="Join Full", command=janela.destroy)
        merge_menu.add_command(label="Left Join", command=janela.destroy)
        merge_menu.add_command(label="Merge Outer", command=janela.destroy)
        menu_bar.add_cascade(label="Merge", menu=merge_menu) 
        
        # CRIA MENU RELATÓRIOS
        relatorio_menu = tk.Menu(menu_bar, tearoff=0)
        relatorio_menu.add_command(label="Consolidar", command=janela.destroy)
        relatorio_menu.add_command(label="Quebrar", command=janela.destroy)
        menu_bar.add_cascade(label="Relatório", menu=relatorio_menu) 
        
        self.janela_principal.config(menu=menu_bar)
        
        # CRIANDO TREEVIEW (ONDE VERIFICA AS TABELAS)
        self.tree = tk.ttk.Treeview(self.janela_principal)
        self.tree.pack(expand=False)

    def soma_colunas_com_valor(self):
        resultados = []
        for coluna in self.df.columns:
            if coluna.upper() == "ID":
                pass
            else:
                if pd.api.types.is_numeric_dtype(self.df[coluna]):
                    valores_numericos = self.df[coluna][1:]
                    valores_numericos = pd.to_numeric(valores_numericos, errors="coerce")
                    valores_numericos = valores_numericos[~np.isnan(valores_numericos)]
    
                    soma = valores_numericos.sum()
                    resultado = f"A soma da coluna {coluna} é {soma}"
                    resultados.append(resultado)

        self.resultado_label.config(text="\n".join(resultados))
    
    def carregar_excel(self):
        
        # DEFINE O TIPO DE ARQUIVO
        tipo_de_arquivo = (("Excel files", "*.xlsx;*.xls"), ("All files", "*.*"))
        
        # ABRE A JANELLA PARA SELECIONAR O ARQUIVO
        self.nome_do_arquivo = filedialog.askopenfilename(title="Selecione o arquivo", filetypes=tipo_de_arquivo)
        
        try:
            self.df = pd.read_excel(self.nome_do_arquivo)
            self.atualiza_treeview()
        except Exception as e:
            messagebox.showerror("Erro", f"Não foi possível abrir o arquivo: {e}")

        self.soma_colunas_com_valor()

    def atualiza_treeview(self):

        #Apaga todo o conteúdo da treeview
        self.tree.delete(*self.tree.get_children())

        # DEFINE AS COLUAS DA TREEVIEW COM BASE NAS COLUNAS DF
        self.tree["columns"] = list(self.df.columns)
        for column in self.df.columns:
            self.tree.heading(column, text=column)

        for i, row in self.df.iterrows():
            values = list(row)
            for j, value in enumerate(values):
                if isinstance(value, np.generic):
                    values[j] = np.asscalar(value)

            self.tree.insert("", tk.END, values=values)

    def renomear_coluna(self):

        # CRIA UMA TELA EM SEGUNDO PLANO
        janela_renomear_coluna = tk.Toplevel(self.janela_principal)
        janela_renomear_coluna.title("Renomear Coluna")

        largura_janela = 400
        altura_janela = 450

        janela_renomear_coluna.configure(bg="#FFFFFF")
        label_coluna = tk.Label(janela_renomear_coluna, text="Digite o nome da coluna que quer renomear: ", font=("Arial", 12), bg="#FFFFFF")
        label_coluna.pack(pady=10)
        entry_coluna = tk.Entry(janela_renomear_coluna, font=("Arial", 12))
        entry_coluna.pack()

        label_novo_nome = tk.Label(janela_renomear_coluna, text="Digite o novo nome: ", font=("Arial", 12), bg="#FFFFFF")
        label_novo_nome.pack(pady=10)
        entry_novo_nome = tk.Entry(janela_renomear_coluna, font=("Arial", 12))
        entry_novo_nome.pack()

        botao_renomear = tk.Button(janela_renomear_coluna,text="RENOMEAR", font=("Arial", 12), command=lambda: self.funcao_renomear_coluna(entry_coluna.get(), entry_novo_nome.get(), janela_renomear_coluna))
        botao_renomear.pack(pady=20)
        
        janela_renomear_coluna.mainloop()

    def funcao_renomear_coluna(self, coluna, novo_nome, janela_renomear_coluna):
        if novo_nome:
            self.df = self.df.rename(columns={coluna: novo_nome})
            self.atualiza_treeview()

            janela_renomear_coluna.destroy()

editor = ExcelEditor(janela)

janela.mainloop()